In this notebook, we will solve the full MHD equations, where the induction equation is coupled to the momentum equation via the Lorentz Force. You are going to use a numerical code that uses a combination of operator splitting, upwind methods, and method of characteristics-constrained transport for MHD.

# Solving the MHD equations using numerical tools - Introduction to FARGO3D

#### Preparations

First we need to create a temporary folder in the virtual machine file system, because the ERDA filesystem where you normally store your files is extremely slow, and not well-suited for compiling large code bases.

In [ ]:
import os
import shutil

In [ ]:
# Get the current working directory
cwd = os.getcwd()
# Go to home folder, which is in the temporary filesystem of the virtual machine
home = '/home/jovyan'
os.chdir(home)

#### Cloning the code from a git repository

First, download the code from its git repository

In [ ]:
!git clone https://bitbucket.org/fargo3d/public.git fargo3d
fargo_folder = os.getcwd()+'/fargo3d'

Next, we need to compile the source files. We will compile the code with an MHD setup called Orszag-Tang Vortex test (Orszag & Tang 1979).

#### Building the code (compilation)

In [ ]:
os.chdir(fargo_folder)
!make clean
!make SETUP=otvortex >& log_compile #This is the Orszag-Tang Vortex test

### Running the code (evolve the MHD equations)

Next, let's run the code from the code directory

In [ ]:
os.chdir(cwd) # change to exercise folder

In [ ]:
os.system('rm -rf fargo3d setups std') # make sure nothing left-over from old runs is present
os.system('cp -f "'+fargo_folder+'/fargo3d" "'+cwd+'"')   # get executable
os.system('cp -rf "'+fargo_folder+'/setups" "'+cwd+'"')   # get setups
os.system('cp -rf "'+fargo_folder+'/std" "'+cwd+'"')      # extra config files

In [ ]:
!./fargo3d -o "ny=128, nz=128, dt=0.05, ntot=10, outputdir=outputs/otvortex" setups/otvortex/otvortex.par

Let's explore the output directory

In [ ]:
ls outputs/otvortex/

The domain of the mesh is stored in the files:
- domain_x.dat
- domain_y.dat (it contains 6 ghost zones (3 at the beginning + 3 at the end, used for boundaries)
- domain_z.dat (it contains 6 ghost zones (3 at the beginning + 3 at the end, used for boundaries)

The hydrodynamics fields are: 
- gasdens%.dat
- gasvx%.dat 
- gasvy%.dat 
- gasvz%.dat

The magnetic fields are: 
- bx%.dat 
- by%.dat 
- bz%.dat

Setup related files are in "fargo3d/setups/otvortex/"

The MHD algorithm is in fargo3d/src/mhd.c

In [ ]:
os.system('cat "'+fargo_folder+'/src/mhd.c"')

### Let's plot some data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
#Some customization
plt.rcParams['figure.figsize'] = (5, 5)
plt.rcParams['image.origin']   = 'lower'

We first create a class that will be useful to obtain the parameters used for each run.

In [ ]:
class Parameters():
    """
    Class for reading the simulation parameters (taken from utils/python/parameters.py).
    input: string -> name of the parfile, normally variables.par
    """
    def __init__(self, directory=''):
        if len(directory) > 1:
            if directory[-1] != '/':
                directory += '/'
        try:
            params = open(directory+"variables.par",'r') #Opening the parfile
        except IOError:                  # Error checker.
            print("Check your directory.")
            return
        lines = params.readlines()     # Reading the parfile
        params.close()                 # Closing the parfile
        par = {}                       # Allocating a dictionary
        for line in lines:             # Iterating over the parfile
            name, value = line.split() # Spliting the name and the value (first blank)
            try:
                float(value)           # First trying with float
            except ValueError:         # If it is not float
                try:
                    int(value)         #                   we try with integer
                except ValueError:     # If it is not integer, we know it is string
                    value = '"' + value + '"'
            par[name] = value          # Filling the dictionary
        for name in par:               # Iterating over the dictionary
            exec("self."+name.lower()+"="+par[name]) # Making the atributes at runtime
            
def get_data(fieldname,n,params):
    """
    We load some scalar field (e.g., density, vx, Bx, etc...)
    and make it 2D (because for this exercise all our fields are 2D).
    """
    return np.fromfile(params.outputdir+fieldname+str(n)+".dat").reshape(params.nz,params.ny)    

Example of use

In [ ]:
params = Parameters("outputs/otvortex/")

# In the notebook, if you now write params and press tab, 
# the autocompletion will show you all the parameters used 
# for the run
density = get_data("gasdens",10,params)
extent  = [params.ymin,params.ymax,params.zmin,params.zmax]
plt.imshow(density,extent=extent)
plt.xlabel("Y")
plt.ylabel("Z")
plt.show()

### Task 1: For the Orszag-Tang Vortex test, plot $B_y$, $B_z$, $V_y$ and $V_z$ for $t=0.5$

In [ ]:
plt.figure(figsize=(10,10))
params = Parameters("outputs/otvortex/")

for i,name in enumerate(["gasvy","gasvz","by","bz"]):
    plt.subplot(2,2,i+1)
    plt.title(name)
    
    #Get data and plot it
    ... 

### Task 2: For the Orszag-Tang Vortex test, plot the streamlines and the magnetic lines 

In [ ]:
#We need the domian
n = 10

vy = ...
vz = ...
by = ...
bz = ...

outputdir = params.outputdir

# We need a mesh with the domain for the task

# First, load the 1D arrays with the domain
y_1d  = np.loadtxt(params.outputdir+"domain_y.dat")[3:-4]
z_1d  = np.loadtxt(params.outputdir+"domain_z.dat")[3:-4]

# Now, create a 2D mesh using meshgrid
y_2d,z_2d = np.meshgrid(y_1d,z_1d)

plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.title("Magnetic lines")
plt.imshow(density,extent=[params.ymin,params.ymax,params.zmin,params.zmax])
... #Magnetic lines here

plt.subplot(1,2,2)
plt.title("Streamlines")
plt.imshow(density,extent=[params.ymin,params.ymax,params.zmin,params.zmax])
... #Streamlines here

### Task 3: Calculate $\nabla \cdot {\bf B}$, plot it with imshow and verify $\nabla \cdot {\bf B} = 0$ for all the mesh. Consider the solution at $t = 0.5$

$\nabla \cdot {\bf B} = \partial_x B_x + \partial_y B_y + \partial_z B_z$, which in finite differences it reads:

In this problem, $B_x = 0$, so:
\begin{equation}
\nabla \cdot {\bf B} = \frac{{B_y}_{jk}-{B_y}_{j-1k}}{\Delta y} + \frac{{B_z}_{jk}-{B_z}_{jk-1}}{\Delta z}
\end{equation}

Tip: remember that $\Delta y$ and $\Delta z$ are constant, so, if the divergence is zero, then you only need to show that:
\begin{equation}
{B_y}_{jk}-{B_y}_{j-1k} + {B_z}_{jk}-{B_z}_{jk-1} = 0
\end{equation}

In [ ]:
n  = 10
by = ... 
bz = ...

divB = # ...
plt.imshow(divB) # ...
plt.colorbar()   # ...

We covered the basics of the code. You can find further documentation in https://fargo3d.bitbucket.io/, and the algorithms used in https://ui.adsabs.harvard.edu/abs/2016ApJS..223...11B/abstract

---------------------

### Linear MRI test

You are going to learn how to use FARGO3D to measure the growth rate of the magnetorotational instability (MRI) and compare it to the values predicted by linear perturbation analysis.

The methods and results of this exercise not only validates the correctness of the solver implemented in a numerical code but also serve as a starting point for studying the so-called nonlinear regime of the MRI.

### Preliminaries

The magnetorotational instability (MRI) can be derived using the shearing-sheet approximation, which consists of a local expansion of the equations around a fiducial radius in a rotating frame within a disk. 

This treatment allows us to neglect the curvature terms and simplify the equations while maintaining important physical properties of the system (like the shear and non-inertial forces in the case of an astrophysical disk).

In order to derive the equations leading to the MRI, we linearize the Navier-Stokes + Induction equations in the incompressible limit ($\nabla \cdot {\bf v}$) = 0 over a constant background magnetic field along the vertical direction $z$ and a constant sheared background (see, e.g. chapter 14.9 from the Theoretical Astrophysics course for more details). We only assume perturbations in the plane.

After writing the perturbations in Fourier space ($\delta \propto \hat{\delta} e^{ik z+\omega t}$), we get the following set of equations for the perturbations:

\begin{align}
\tilde{\omega} \delta v_x  &= 2 \delta v_y + i\tilde{k}\delta b_x, \nonumber \\
\tilde{\omega} \delta v_y  &= -\frac{1}{2} \delta v_x + i\tilde{k} \delta b_y, \nonumber \\
\tilde{\omega} \delta b_x &= i\tilde{k}\delta v_x, \nonumber \\
\tilde{\omega} \delta b_y  &= -\frac{3}{2} \delta b_x + i\tilde{k} \delta v_y,\nonumber \\ \nonumber 
\end{align}

where $\delta b_{x,y}$ = $\delta B_{x,y}/\sqrt{\mu_0\rho}$, $\tilde{k} = k v_A/\Omega_0$ with $v_A = B_z/\sqrt{\mu_0\rho}$ the Alfvén speed and $\tilde{\omega} = \omega/\Omega_0$. For simplicity, we have omitted the "hat" symbols, but remember that perturbations correspond to the Fourier components (which are in general complex quantities).

By defining the vector $\vec{\delta} \equiv [\delta v_x, \delta v_y, \delta b_x, \delta b_y]$, the system can be written as:

\begin{equation}
L\delta = \tilde{\omega} \delta
\end{equation}
with $L$ given by:

\begin{equation}
L=
\begin{bmatrix}
0 & 2 & i\tilde{k} & 0 \\
-1/2 & 0 & 0 & i\tilde{k} \\
i\tilde{k} & 0 & 0 & 0 \\
0 & i\tilde{k} & -3/2 & 0 \\
\end{bmatrix},
\end{equation}

In this way, we have reduced the problem to an eigenvalue problem (whose solution gives us the dispersion relation  of the problem, i.e., the pair $k,\omega$ such that non-trivial solution for the perturbations exists. 

#### Dispersion relation

The characteristic equation ($\det(L-\omega I) = 0$) is:

\begin{equation}
\omega^4+\left(2\tilde{k}^2 + 1\right) \omega^2 + \tilde{k}^2\left(\tilde{k}^2 - 3 \right) = 0 .
\end{equation}

This is a parabola in $\omega^2$ and the (4) roots are:

\begin{equation}
\omega = \pm\left[ -\frac{1 + 2\tilde{k}^2}{2} \pm \frac{1}{2}\sqrt{1+16\tilde{k}^{2}}\right]^{1/2}.
\end{equation}

One of these eigenvalues is real and positive (under conditions you can easily determine), leading to an exponential growth of the perturbations. This exponential growth is called the Magnetorotational instability. 

The unstable eigenvalue is:

\begin{equation}
{\omega_{\rm unstable}} = \left[ -\frac{1 + 2\tilde{k}^2}{2} + \frac{1}{2}\sqrt{1+16\tilde{k}^{2}}\right]^{1/2}
\end{equation}

### Task 1 

- Plot the dispersion relation of the instability (i.e., growth rate ($\omega_{\rm unstable}$) as a function of $\tilde{k}$ (assume $\tilde{k}>0$).
    
- Show that the maximum growth rate occurs at $\tilde{k}_\text{max} = \sqrt{15}/4$ and prove that it is equal to $\omega_\text{max} = 3/4$.
    
- Show that there is a cut-off at $\tilde{k}_\text{cut-off} = \sqrt{3}$ (i.e., for $\tilde{k}>\tilde{k}_{\rm cut-off}$, $\omega_{\rm unstable}$ is not real anymore).

In [ ]:
...

### Evolution of an isolated mode

Once the eigenvalues of the operator $L$ are known, the eigenvectors can be obtained (for each eigenvalue), and the linear solution of the system, for each particular mode, can be written as:
\begin{align}
\delta v_x(z,t)  &= \frac{A}{N} \tilde{\omega} \sin{\left(kz\right)} e^{\omega t} \nonumber \\
\delta v_y(z,t)   &= \frac{A}{N} \frac{\tilde{\omega}^2 + \tilde{k}^2}{2} \sin{\left(kz\right)} e^{\omega t} \nonumber  \\
\delta b_x(z,t)  &= \frac{A}{N}\tilde{k} \cos{\left(kz\right)} e^{\omega t} \nonumber \\
\delta b_y(z,t)  &= -\frac{A}{N} \frac{2\tilde{k}\tilde{\omega}}{\tilde{\omega}^2 + \tilde{k}^2} \cos{\left(kz\right)} e^{\omega t} \nonumber 
\end{align}

where $N$ is a normalization factor for the eigenvectors, given by 

\begin{equation}
N = \sqrt{\tilde{\omega}^2 + \left(\frac{\tilde{\omega}^2 + \tilde{k}^2}{2}\right)^2 + \tilde{k}^2 + \left(\frac{2\tilde{k}\tilde{\omega}}{\tilde{\omega}^2 + \tilde{k}^2}\right)^2}
\end{equation} 

The initial amplitude of the perturbations is given by $A$. (Note: the solution for any arbitrary initial condition is can now be obtained as the linear combination of these four modes)


### Task 2 - Using FARGO3D, measure the linear growth rate $\omega$ of the MRI for different values of $\tilde{k}$

Now we will recover the evolution of the unstable mode by solving the full set of nonlinear MHD equations from very small perturbations, so the linear approximation remains valid. In other words, the goal of the exercise is to use FARGO3D to build the dispersion relation numerically, exciting the unstable mode to measure the corresponding growth rate.

### Guideline

#### Step 1 - Incorporating a new setup

Copy the file "mri-growth-sb.zip" to fargo3d/setups/.
Opening the Terminal you can unpack the file using the command "unzip FILENAME". Once you have unzip'ed the file, you can compile the code with the MRI setup.

**remember** that the file should be copied to the temporary folder where the full fargo-3d code resides. That is in /home/jovyan/fargo. From the terminal something like

`cp mri-growth-sb.zip /home/jovyan/fargo3d/setups`

`cd /home/jovyan/fargo3d/setups`

`unzip mri-growth-sb.zip`

should do the trick. **The cell below accomplishes the same, but verify that it has been done. Last line should be something like "inflating: mri-growth-sb/mri-growth-sb.bound.0"**

In [ ]:
os.system('cp -f "'+cwd+'/mri-growth-sb.zip" "'+fargo_folder+'/setups/" ')   # copy MRI setup
os.chdir(fargo_folder+'/setups') # change to setup folder
os.system('unzip mri-growth-sb.zip')   # unzip it
os.chdir(cwd) # change to exercise folder

We first clean the code (remove the executable and compiled sources that were used for the previous setup)

In [ ]:
os.chdir(fargo_folder)
!make clean

And compile the code again, but with the new setup "mri-growth-sb"

In [ ]:
!make SETUP=mri-growth-sb > build_mri_log

Before running the code, have a look at the initial condition file (setups/mri-growth-sb/condinit.c) and the parameters file (setups/mri-growth-sb/mri-growth-sb.par), which allows you to modify the parameters of the run.

Any parameter there can also be changed at execution time through the flag "-o",

e.g.: 
           
    ./fargo3d -o "dt=0.1 nx=32 ny=32" setups/mri-growth-sb/mri-growth-sb.par

#### Comment about the initial condition
	
In the file "condinit.c", the eigenvectors are initialized proportional to trigonometric functions with argument $\left(2\pi z/\Delta Z\right)$, with $\Delta Z = Z_\text{max}-Z_\text{min}$. This is equivalent to considering the wave number $k = 2\pi/\Delta Z$. So, in order to change the excited mode, it is enough to change the vertical size of the box.

The size $\Delta Z$ of the box is related to the wave number $k$ and the parameters of the problem by the following relation:

\begin{equation}
k = \frac{2\pi}{\Delta Z} = \frac{\tilde{k} \Omega_0}{v_{\rm A}} \Longrightarrow \Delta Z = \frac{2\pi v_{\rm A}}{\tilde{k}  \Omega_0}.
\end{equation}

With the default values provided in the setup, $v_A \simeq 0.003162$, so the maximum growth rate occur (at $\tilde{k}_{\text{max}} = \sqrt{15}/4$) at:

\begin{equation}
\Delta Z_\text{max} \simeq 0.0205 
\end{equation}
	
which is the default "Zmax" value in the parameters file. 

Thus, in order to measure the dispersion relation, we will only modify the vertical extension of the box Zmax.

------------

#### Step 2 - Running the code with the new setup

In [ ]:
os.chdir(fargo_folder)
!make #just in case you have modified some source file
os.chdir(cwd) # change to exercise folder
os.system('rm -rf fargo3d setups std') # make sure nothing left-over from old runs is present
os.system('cp -f "'+fargo_folder+'/fargo3d" "'+cwd+'"')   # get executable
os.system('cp -rf "'+fargo_folder+'/setups" "'+cwd+'"')   # get setups
os.system('cp -rf "'+fargo_folder+'/std" "'+cwd+'"')      # extra config files
!./fargo3d -o "dt=0.1 zmax=0.0205 ny=32 nz=32" setups/mri-growth-sb/mri-growth-sb.par

#### Step 3 - For fixed $y_0$, plot $B_x(y_0,z)$ for output numbers 5, 10, and 15 (in the same axis). Explain what you observe.
 
Let's concentrate on the solution for the component $B_x$, which is:

\begin{align}
\delta b_x(z,t) \propto \cos{\left(kz\right)} e^{\omega t} \nonumber \\
\end{align}

Note that for this task, the proportionality factors are not important. Since the solution does not depend on $y$, we can just plot a cut along the $z$-direction. Furthermore, we can also anticipate what we should observe: a trigonometric function whose amplitude grows with time exponentially.

Example of cut:

    get_data("bx",3,params)[:,params.ny//2]

In [ ]:
...

---------------

#### Step 4 - Using $B_x$, measure the growth-rate $\omega$ for the results from step2. Plot the value measured on top of the dispersion relation obtained analitically.

Tip 1:

The solution is of the form:
\begin{align}
\delta b_x(z,t) \propto \cos{\left(kz\right)} e^{\omega t} \nonumber
\end{align}

so, if we fix $z=z_0$:
\begin{align}
\delta b_x(z_0,t) = C e^{\omega t} \nonumber
\end{align}

with $C$ a constant, i.e., the perturbation must growth exponentially. 

Now, we can calculate:

\begin{equation}
\log|\delta b_x (z_0,t)| = \omega t + \log(C)
\end{equation}

Previous expression shows that we can measure the growth rate by plotting $\log(\delta b_x)$ ($B_x$ in our setup) as a function of time, which is a straight line. Furthermore, we can use a simple linear regresion to determine the slope (growth rate) from our data. (Check the function np.polyfit())

Tip 2: The time corresponding to each output can be obtained as:

    time = params.dt * params.ninterm * i 

where i is the output number

In [ ]:
params = Parameters("outputs/mri-growth-sb/")

# We store the data in two lists    
bx   = []
time = []

for i in range(20):
    time.append(...
    by.append(...

# We convert the lists to arrays so we can use numpy to modify them (if needed)
time = np.array(time)
bx   = np.array(bx)

# And plot it
plt.plot(time,np.log(np.abs(bx)))
plt.show()

Now, we can use np.polyfit to obtain the slope of the curve

In [ ]:
# Polyfit gives us the coefficients of the polynomial a*x+b
a,b = np.polyfit(...

x = np.linspace(time.min(),time.max(),100) # We create a domain array
y = a*x+b                                  # This is our fit

# And finally compare our fit with the data
plt.plot(time,np.log(np.abs(by)),'o',label='data')
plt.plot(x,y,'k--',label='fit')
plt.legend()
plt.show()

As we can see, the fit is excellent! Now, we can plot the measured growth rate (a) on top of our analytical dispersion relation:

In [ ]:
# This is the dispersion relation

k_tilde = np.linspace(0,np.sqrt(3),100)
omega_unstable = np.sqrt(-0.5*(1 + 2*k_tilde**2) + 0.5*np.sqrt(1+16*k_tilde**2))
plt.plot(k_tilde,omega_unstable)

# And now we plot the measured value
k_tilde_experiment = np.sqrt(15)/4.0
omega_experiment = a

plt.plot(k_tilde_experiment,omega_experiment,'ko')


#### Step 5 - Determine the growth rate for all these values: 

$\Delta Z$ = [0.0911,  0.0514,  0.0358,  0.0274 ,	0.0223,  0.0188,  0.0162,  0.0142 ,  0.0127] and plot the pairs (${\tilde k},\omega$) for all the cases in one figure, on top of the analytical solution (as done above).

For each run, you can obtain the corresponding $\tilde{k}$ from the terminal or by using the formula that was used to deduce the $\Delta Z$ corresponding to the maximum growth rate. Tip: in order to save time (and minimize the probablity of typos), the lines needed to run the code can be generated automatically using strings, as done below.

In [ ]:
Dz = [0.0911, 0.0514, 0.0358, 
      0.0274, 0.0223, 0.0188, 
      0.0162, 0.0142, 0.0127]

# If you have time, you can play with different number of cells
# but the default for the exercise is 32.
ny      = 32  #Number of cells in the radial direction
nz      = 32  #Number of cells in the vertical direction
dt      = 0.2 #Time between outputs

for i,delta_z in enumerate(Dz):
    arguments = '"zmax={0:4.4f} outputdir=outputs/output{1:d} ny={2:d} nz={3:d} dt={4:2.2f}"'.format(delta_z,i, ny, nz, dt)
    exec_line = "!./fargo3d -o "+arguments+" setups/mri-growth-sb/mri-growth-sb.par > log{0:d}".format(i)
    print(exec_line)

### __Absalon turn-in__

With all the file processing and simulation running, this notebook can become unweildy. Therefore copy central code, plots and comment for each task to a PDF report. Either by actually writing text or (for the lazy) by screenshotting cells

Upload the pdf report with solutions together with the notebook and a pdf of the notebook.